# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.234309791452                   -0.50    7.0         
  2   -7.250263068647       -1.80       -1.41    1.0   7.78ms
  3   -7.251160053968       -3.05       -2.22    1.0   7.86ms
  4   -7.251307589008       -3.83       -2.29    2.0   9.92ms
  5   -7.251333860480       -4.58       -2.67    1.0   8.11ms
  6   -7.251337184615       -5.48       -3.05    1.0   8.42ms
  7   -7.251338696688       -5.82       -3.59    1.0   8.32ms
  8   -7.251338790016       -7.03       -4.04    2.0   10.1ms
  9   -7.251338797709       -8.11       -4.54    2.0   10.8ms
 10   -7.251338798445       -9.13       -4.88    2.0   9.84ms
 11   -7.251338798689       -9.61       -5.66    2.0   10.4ms
 12   -7.251338798704      -10.83       -6.09    3.0   12.2ms
 13   -7.251338798704      -12.38       -6.51    1.0   8.85ms
 14   -7.251338798705      -12.94       -6.84    2.0   10.3ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.04821245375055228
[ Info: Arnoldi iteration step 2: normres = 0.4876366971841036
[ Info: Arnoldi iteration step 3: normres = 0.9594391021368813
[ Info: Arnoldi iteration step 4: normres = 0.36363451350747944
[ Info: Arnoldi iteration step 5: normres = 0.4784622136594223
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (5.26e-02, 7.92e-02, 4.13e-01, 2.19e-01, 3.30e-02)
[ Info: Arnoldi iteration step 6: normres = 0.2312161140972307
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.08e-02, 1.32e-01, 1.55e-01, 9.46e-02, 1.51e-02)
[ Info: Arnoldi iteration step 7: normres = 0.07238706065950778
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (3.54e-04, 7.86e-03, 8.50e-03, 2.29e-02, 6.29e-02)
[ Info: Arnoldi iteration step 8: normres = 0.07700761994350452
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.15e-